<a href="https://colab.research.google.com/github/garciafranciscomartn/nextflow_intro_rsg/blob/main/Modulo2_Input_Output_Channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input, Output y *Channels*

¿Qué son los Canales (*Channels*)?

Los canales son las estructuras de datos fundamentales en Nextflow que permiten el flujo de información entre procesos. Son como "tuberías" que transportan datos de un proceso a otro de manera asíncrona.

Características principales:

*   Son inmutables: una vez creados, no se pueden modificar
*   Son asíncronos: los datos fluyen cuando están disponibles
*   Conectan procesos automáticamente
*   Pueden contener cualquier tipo de dato (archivos, valores, listas, etc.)

Fábricas de Canales ([*Channel Factories*](https://www.nextflow.io/docs/stable/reference/channel.html#)
Las fábricas de canales son métodos especiales que crean canales desde diferentes fuentes de datos, algunos ejemplos son los siguientes:

*   Channel.of() - Crea canales desde valores específicos
*   Channel.fromPath() - Crea canales desde archivos/rutas, y es el que usamos aquí
*   Channel.fromFilePairs() - Para archivos paired-end

Estos son algunos de los tipos, pero si quieren mas información hay un archivo en el directorio donde estamos que se llama fabricas_canales.pdf donde les deje una breve explicacion de los diferentes tipos, sus usos y diferencias entre si.

Sigamos con el módulo, nuestro input entonces quedo definido así:
```
input_ch = Channel.fromPath("test_1.fastq.gz")
```

Channel.fromPath() permite buscar archivos que coincidan con el patrón especificadom y crea un canal con las rutas de los archivos encontrados.
Tiene la ventaja de aceptar patrones globales como *.fastq.gz o data/*.txt, y al usarlo este verifica la existencia de archivos por defecto

En nuestro ejemplo, busca específicamente el archivo test_1.fastq.gz, crea un canal que contiene la ruta a ese archivo y por último el canal se pasa al proceso runFastQC como entrada.

Después de entender cómo se crean y manejan los canales en el workflow, es importante comprender cómo los procesos reciben y utilizan estos datos.

En nuestro proceso runFastQC, el input se define así:

```

input:
path fastq_file

```

¿Qué significa esto?

-   path: Es el tipo de input que especifica que esperamos recibir una ruta de archivo
-   fastq_file: Es el nombre de la variable que usaremos dentro del proceso para referenciar el archivo

Los procesos pueden recibir diferentes tipos de input:

-   path: Para archivos y directorios
-   val: Para valores simples (números, strings, booleanos)
-   tuple: Para múltiples elementos agrupados

Son algunos de los tipos de input que puede recibir, para mayor detalle sobre los inputs se pueden referir a la documentacion de nexflow sobre [Inputs](https://www.nextflow.io/docs/latest/process.html#inputs)

Es importante destacar nombres de las variables en el proceso y en el workflow NO necesitan ser iguales.

En nuestro ejemplo:

En el workflow: input_ch (nombre del canal)
En el proceso: fastq_file (nombre de la variable del input)

El workflow maneja la lógica de flujo de datos, mientras que el proceso se enfoca en la tarea específica
Esto permite la reutilizacion ya que un mismo proceso puede ser llamado desde diferentes workflows con canales que tengan nombres distintos
Y por ultimo mejora la claridad semántica ya que cada contexto puede usar nombres más descriptivos para su propósito específico

Una vez definido el input, la variable fastq_file se puede usar en la sección script:

```
script:
"""
fastqc ${fastq_file}
"""
```

Aquí, ${fastq_file} se expande al path real del archivo que llegó a través del canal.

-   Channel.fromPath("test_1.fastq.gz") crea un canal con la ruta del archivo

-   runFastQC(input_ch) pasa el canal al proceso
-   path fastq_file recibe el dato del canal y lo asigna a la variable fastq_file
-   El script usa ${fastq_file} para acceder al archivo real
Esta flexibilidad en el nombrado permite que los procesos sean más modulares y reutilizables en diferentes contextos.

Sigamos por la definición del output

### Output

Estructura del Output:
*   output: - Palabra clave que define qué archivos produce el proceso
*   path - Tipo de dato que especifica que son archivos/directorios
*   "*.html" y "*.zip" - Patrones glob que capturan archivos

Desglose de cada línea:
   -   path "*.html"
       Busca: Todos los archivos que terminen en .html en el directorio de trabajo del proceso
       Captura: test_1_fastqc.html
       Tipo: Archivos individuales
   -   path "*.zip"
       Busca: Todos los archivos que terminen en .zip
       Captura: test_1_fastqc.zip, test_2_fastqc.zip, etc.
       Tipo: Archivos comprimidos con datos detallados

¿Por qué dos líneas separadas?

Cada línea crea un canal de salida diferente
Permite que otros procesos consuman solo HTML o solo ZIP según necesiten
Mayor flexibilidad en el pipeline

**Aclaración**:La definición de salida no determina qué salida (output) se creará. Simplemente declara cuál es la salida esperada, para que Nextflow pueda buscarla una vez que se complete la ejecución. Esto es necesario para verificar que el comando se ejecutó correctamente y para pasar la salida a procesos posteriores si es necesario. Las salidas producidas que no coincidan con lo declarado en el bloque de salida no se pasarán a procesos posteriores.

---
                                                                                                                                                        
Ahora vamos a empezar a modificar el pipeline para ver diferentes capacidades y herramientas de nextflow, comencemos con el ouput. La salida producida por nuestro pipeline está enterrada en un directorio de trabajo a varios niveles de profundidad. Esto se hace a propósito: Nextflow controla ese directorio y no se espera que interactuemos directamente con él.

Sin embargo, eso hace que sea poco práctico recuperar los outputs que realmente nos interesan.

### Manejar destino de output

Afortunadamente, Nextflow ofrece una forma más conveniente de manejar esto, llamada directiva *publishDir*, que actúa a nivel de proceso. Esta directiva le indica a Nextflow que publique el(los) output(s) del proceso en un directorio de salida designado. Por defecto, las salidas se publican como enlaces simbólicos desde el directorio work. Esto nos permite recuperar el archivo de salida deseado sin tener que buscarlo manualmente dentro del directorio de trabajo.

En el archivo de script del flujo de trabajo hola_fastqc_completo.nf, realizá la siguiente modificación en el código:
```
process runFastQC {

    publishDir 'results', mode: 'copy'

    output:
        path 'output.txt'

```
Guardamos el archivo y lo volvemos a ejecutar

```
    
nextflow run hola_fastqc.nf

```
    
Esta vez, Nextflow ha creado una nueva carpeta llamada results/. Nuestros archivos de output estan en esta carpeta. Si revisás su contenido, debería coincidir con la salida en las subcarpetas work. Esta es la forma conveniente en que publicamos archivos de resultados fuera de las carpetas de trabajo.

Cuando estás trabajando con archivos muy grandes que no necesitás conservar por mucho tiempo, podés preferir configurar la directiva **publishDir** para que cree un enlace simbólico al archivo en lugar de copiarlo. Sin embargo, si eliminás el directorio work como parte de una operación de limpieza, perderás acceso al archivo. Por eso, asegurate siempre de tener copias reales de todo lo que te importa antes de borrar nada. Se ha propuesto una nueva opción de sintaxis que permite declarar y publicar salidas a nivel de flujo de trabajo. Esto eventualmente hará que el uso de publishDir a nivel de proceso sea redundante para pipelines ya finalizados. Sin embargo, se espera que publishDir siga siendo muy útil durante el desarrollo de pipelines.

Ahora que entendemos el input y el ouput vamos va trabajar sobre las variables.

### Usar una variable por línea de comandos
En su estado actual, nuestro flujo de trabajo utiliza el archivo fastq codificado directamente en el comando del proceso. Queremos agregar algo de flexibilidad usando una variable de entrada, para poder cambiar el archivo fácilmente en tiempo de ejecución.
Esto requiere que configuremos un parámetro de línea de comandos y proporcionarlo como input a la llamada del proceso. Nextflow tiene un sistema de parámetros incorporado llamado params, que facilita declarar y usar parámetros CLI. La sintaxis general es params.<nombre_parametro>, lo que le indica a Nextflow que espere un parámetro --<nombre_parametro> en la línea de comandos.

Queremos crear un parámetro llamado --fastq, así que declaramos params.fastq y lo pasamos al proceso:
```
    // Creamos un canal con el input de archivos FASTQ
    input_ch = Channel.fromPath(params.fastq))

```
Y ejecutamos el script pasando el parametro --fastq con la ruta hacia el archivo
```
nextflow run hola_fastqc.nf --fastq test_1.fastq.gz
```
Ahora bien, si no le pasamos el parametro --fastq al ejecutar el script da ERROR ya que no tiene ningun valor por default. Vamos a setear un valor por default para fastq, que si bien en este caso no es tan relevante al manejar pipelines es util no tener que especificar todas las variables en cada ejecución.
Modificamos en el comienzo del script (debajo del *shebang*)
```
/*
 * Parámetros
 */
params.fastq = "test_1.fastq.gz"
```
Y probamos correr nuestros script sin pasarle el parámetro [--fastq]
¡Funciono!
Antes de seguir vamos a modificar el parametro output asi la carpeta generada tiene un valor por default pero si quisieramos podriamos cambiar el destino en el cual publishDir copia los ouputs
El script quedaria
```
/*
 * Parámetros
 */
params.fastq = "test_1.fastq.gz"
params.outdir = "results"

process runFastQC {
    container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8"

    publishDir "${params.outdir}, mode: 'copy'
```

**Truco sobre sintaxis**
* Los parámetros de Nextflow (como -resume) usan un solo guion (-).
* Los parámetros del pipeline (como --greeting) usan doble guion (--).

### ¿Que pasa si queremos trabajar con mas de un input?

Para eso vamos a comenzar descargando mas archivos en nuestra carpeta

```
wget -c https://raw.githubusercontent.com/nf-core/test-datasets/modules/data/genomics/homo_sapiens/illumina/fastq/test_2.fastq.gz
```
Y corramos el script usando el parametro --fastq con un patrón global ["*.fastq.gz"]

Deberiamos ver algo así:

```
executor >  local (2)
[8d/215646] runFastQC (1) | 2 of 2 ✔

```

Veamos en la carpeta generada [8d/215646] los outputs generados:

```
tree -a work/8d

work/8d
└── 215646cc4d87baec49e78aa05b5c62
    ├── .command.begin
    ├── .command.err
    ├── .command.log
    ├── .command.out
    ├── .command.run
    ├── .command.sh
    ├── .exitcode
    ├── test_2.fastq.gz -> /home/fg47909/nextflow_rsg/test_2.fastq.gz
    ├── test_2_fastqc.html
    └── test_2_fastqc.zip
```
Vemos que se indican 2 de 2 llamadas para el proceso, lo cual es alentador, pero esto solo nos muestra una única ejecución del proceso (para el archivo test_2 en este caso), con una ruta de subdirectorio. Por defecto, el sistema de registro ANSI escribe los registros de múltiples llamadas al mismo proceso en la misma línea. Pero podemos desactivar ese comportamiento para ver la lista completa de llamadas al proceso. Para expandir el registro y mostrar una línea por cada llamada al proceso, agrega -ansi-log false al comando:
```
nextflow run hola_fastqc.nf -ansi-log false

```

Esta vez vemos las dos ejecuciones del proceso y sus subdirectorios de trabajo asociados listados en la salida. Queda despues en uno evaluar si tener la lista de todos los outputs es necesario o no, dependiendo de la cantidad de elementos que tenga el flujo de trabajo.

Ahora vamos a explorar algunos operadores que nos permitan mejorar nuestro script, para eso descarguemos dos archivos más:
```
wget -c https://raw.githubusercontent.com/nf-core/test-datasets/modules/data/genomics/homo_sapiens/illumina/fastq/test2_1.fastq.gz

wget -c https://raw.githubusercontent.com/nf-core/test-datasets/modules/data/genomics/homo_sapiens/illumina/fastq/test2_2.fastq.gz

```

### Operadores
Muchas veces se da que queremos correr multiples inputs y estos valores estan contenidos en un archivo. Por ejemplo, podemos tener un archivo fastq.csv que tenga el path de un fastq en cada linea. Necesitamos modificar nuestro workflow para que pueda leer los valores de un archivo como ese.
Primero vamos a crear ese archivo con los path hacia nuestros fastq
```
ls test*.fastq.gz > fastq.csv
```

Vamos a modificar el tipo de canal para que pueda manejar un archivo y para esto vamos a usar los operadores. Nextflow permite aplicar estos operadores a los canales para transformar sus contenidos antes de pasarlos a un proceso.

Los primeros operadores que vamos a ver son view() y splitCsv().

**Operador view**
El operardor [view()](https://www.nextflow.io/docs/latest/reference/operator.html#view) es muy útil para utilizar en procesos de *debugging*, vendria a cumplir la misma funcion que print() en python.

**Operador splitCsv**
El operador [splitCsv()](https://www.nextflow.io/docs/latest/reference/operator.html#splitCsv) sirve para  procesar archivos CSV y convertirlos en canales de datos estructurados.
Para aplicarlo, hay que aplicarselo a la linea del canal.
```
input_ch = Channel.fromPath(params.fastq)
        .view { filename -> "Before splitCsv: $filename" }
        .splitCsv()
        .view { filename -> "After splitCsv: $filename" }

```
**Aclaración** - filename es simplemente el nombre de la variable que elegimos para representar cada elemento que pasa por el canal dentro del operador

Ahora, probemos volver a correr el pipeline
```
nextflow run hola_fastqc.nf --fastq fastq.csv

```
**¡¡ERROR!!**
```

Before splitCsv: /home/fg47909/nextflow_rsg/fastq.csv
After splitCsv: [test2_1.fastq.gz]
After splitCsv: [test2_2.fastq.gz]
After splitCsv: [test_1.fastq.gz]
After splitCsv: [test_2.fastq.gz]
ERROR ~ Error executing process > 'runFastQC (1)'

Caused by:
  Not a valid path value: 'test2_1.fastq.gz'
```

Esta vez Nextflow sí analiza el contenido del archivo, pero cada fastq aparece entre corchetes ([test_2.fastq.gz], etc.), porque splitCsv() convierte cada línea en un elemento de array y runFastQC espera un path.
Vamos a solucionarlo usando otro operador.

Si recorremos la [lista de operadores](https://www.nextflow.io/docs/latest/reference/operator.html) vamos a encontrar el elemento [flatten](https://www.nextflow.io/docs/latest/reference/operator.html#flatten) que hace lo que necesitamos en este caso, flatten() convierte estructuras anidadas en elementos individuales.

**Operador flatten()**
```
.flatten()
.view { filename -> "Procesando archivo: $filename" }
```
**¡¡ERROR!!**
El problema es que estás leyendo los archivoscomo *strings* y no como *paths*. Al usar Channel.fromPath() con un archivo CSV y luego procesarlo, obtienes *strings* de nombres de archivo en lugar de *paths* de archivo válidos.
Para solucionar esto vamos a agregar otro operador mas, el operador map().

**Operador map**
El operador [map()](https://www.nextflow.io/docs/latest/reference/operator.html#map) permite realizar aplicar funciones sobre los elementos del canal, lo que nos posbilita transforar y extraer dats y hasta crear metadatos. En este caso lo vamos a usar para transformar cada elemento del canal aplicando una función file que nos permite buscar el path completo del archivo. Para eso vamos a agregarlo en la definición del canal:

```
    // Creamos un canal con el input de archivos FASTQ
    input_ch = Channel.fromPath(params.fastq)
        .view { filename -> "Before splitCsv: $filename" }
        .splitCsv()
        .view { filename -> "After splitCsv: $filename" }
        .flatten()
        .map { filename -> file(filename)}
        .view { filename -> "Procesando archivo: $filename" }
```

Ejecutamos el script nuevamente
```
[fd/6c59fc] runFastQC (4) | 4 of 4 ✔
Before splitCsv: /home/fg47909/nextflow_rsg/fastq.csv
After splitCsv: [test2_1.fastq.gz]
After splitCsv: [test2_2.fastq.gz]
After splitCsv: [test_1.fastq.gz]
After splitCsv: [test_2.fastq.gz]
Procesando archivo: /home/fg47909/nextflow_rsg/test2_1.fastq.gz
Procesando archivo: /home/fg47909/nextflow_rsg/test2_2.fastq.gz
Procesando archivo: /home/fg47909/nextflow_rsg/test_1.fastq.gz
Procesando archivo: /home/fg47909/nextflow_rsg/test_2.fastq.gz
```
¡Funciono!
Ahora podriamos eliminar los pasos de view para que la ejecución sea mas limpia en la consola (aunque eso depende se sus usos y costumbres)

El uso de operadores es muy amplio, donde hay más de 50 operadores diferentes, divididos en 7 categorías principales que te permiten manipular canales de diferentes maneras:

1. Operadores de filtrado (Filtering operators)
2. Operadores de transformación (Transforming operators)
3. Operadores de división (Splitting operators)
4. Operadores de combinación (Combining operators)
5. Operadores de bifurcación (Forking operators)
6. Operadores matemáticos (Maths operators)
7. Otros operadores (Other operators)

Para un uso más preciso pueden leer la documentación sobre los mismos a la hora de usarlos.

#### Eliminar directorios de trabajo antiguos

Durante el desarrollo de un pipeline, es común ejecutarlo muchas veces mientras se realizan pruebas o ajustes. Esto genera una gran cantidad de archivos dentro del directorio `work/`, distribuidos en múltiples subdirectorios con nombres aleatorios.
Como esos nombres no indican cuándo fueron creados, es difícil saber cuáles corresponden a ejecuciones viejas.Nextflow incluye el subcomando `clean`, que permite borrar automáticamente los subdirectorios de ejecuciones anteriores que ya no necesitamos, con distintas opciones de control.
Para borrar los directorios de trabajo anteriores a una ejecución en particular, usamos:

```
nextflow clean -before marvelous_mestorf -n

```
El flag -n es un modo de prueba que simula lo que se eliminaría, sin borrar nada realmente. Si no ves ninguna salida, puede que el nombre de ejecución no sea válido o que no haya ejecuciones anteriores para borrar.

Si la salida es la esperada y querés proceder con la limpieza, ejecutamos

```

nextflow clean -before golden_cantor -f


```

Esto eliminará efectivamente los directorios mostrados en el paso anterior.